In [1]:
# Download the models

import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2591, done.
remote: Counting objects: 100% (2591/2591), done.
remote: Compressing objects: 100% (2259/2259), done.
remote: Total 2591 (delta 504), reused 1441 (delta 295), pack-reused 0
Receiving objects: 100% (2591/2591), 124.11 MiB | 4.76 MiB/s, done.
Resolving deltas: 100% (504/504), done.


In [2]:
# Compile protobufs and install the object_detection package

In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [4]:
%%bash 
cd models/research
pip install .

Processing /home/tanmay/Github/tf2_adv_ops/Special Projects/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1016586 sha256=6ee11520e65cc1f4d27685ba4c51f7dcab03ae4dd131b0bae584a9b8f5efce20
  Stored in directory: /tmp/pip-ephem-wheel-cache-g2h50xdi/wheels/31/86/35/c824b10ce8d3b7e4b2da57ac1b62066de6b88084e13b9711c4
Successfully built object-detection


In [5]:
import os
import sys
import tarfile
import zipfile

import numpy as np
import tensorflow as tf
import six.moves.urllib as urllib


from PIL import Image
from io import StringIO
from IPython.display import display
from collections import defaultdict
from matplotlib import pyplot as plt

In [6]:
# Get helper funcs from object_detection module

from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [7]:
# Patches

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Prepare Model

In [8]:
# Load Model

def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname = model_name, 
    origin = base_url + model_file,
    untar = True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [9]:
# List of the strings that is used to add correct label for each box.

PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name = True)

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.

PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS